# **【Python】EDINET API を利用して企業の決算書データ(XBRL)を分析フォーマット(CSV)に出力するコード**  
[本コードの説明など](https://cdle.jp/blogs/09e79f64876f)  
  
EDINET APIの利用にあたっては[EDINET API利用規約](https://disclosure.edinet-fsa.go.jp/EKW0EZ0015.html)を確認の上、特に、
> 利用規約の第５条（禁止事項）  
> １．利用者は、以下に掲げる行為を行ってはならないものとします。  
> (2) 短時間における大量のアクセスその他の本機能の運用に支障を与える行為  

で規定される行為は行わないようご留意ください。  
  
個人が趣味で作成したものですので、データやプログラムコードの正確性などは保証できません。  

# SETUP



In [1]:
import os
# --------------------
# フォルダ作成用の環境判定
# --------------------
if 'COLAB_BACKEND_VERSION' in set(os.environ.keys()):
  Colaboratory_env = True
else:
  Colaboratory_env = False

# --------------------
# プロジェクトフォルダの作成
# --------------------
if Colaboratory_env:
  PROJECT = 'EDINET_API'
  # CREATE PROJECT FOLD
  if not os.path.exists(f'/content/{PROJECT}'): 
    !mkdir {PROJECT}
  # CREATE INPUT FOLD 
  if not os.path.exists(f'/content/{PROJECT}/input'): 
    !mkdir {PROJECT}/input
  # CREATE OUTPUT FOLD 
  if not os.path.exists(f'/content/{PROJECT}/output'): 
    !mkdir {PROJECT}/output
  # CREATE MODEL FOLD 
  if not os.path.exists(f'/content/{PROJECT}/model'): 
    !mkdir {PROJECT}/model
  # API DATA FOLD 
  if not os.path.exists(f'/content/{PROJECT}/APIdata'): 
    !mkdir {PROJECT}/APIdata
  
  BaseDir = os.path.abspath(f'/content/{PROJECT}')
  InputDir = os.path.abspath(f'/content/{PROJECT}/input')
  OutputDir = os.path.abspath(f'/content/{PROJECT}/output')
  ModelDir = os.path.abspath(f'/content/{PROJECT}/model')
  APIDir = os.path.abspath(f'/content/{PROJECT}/APIdata')

# --------------------
# 必要であればGoogleDriveをマウント
GoogleDriveMount=False
# --------------------
# if Colaboratory_env:
#   from google.colab import drive 
#   drive.mount('/content/drive')
#   GoogleDriveMount=True
#   CopyGoogleDriveDir='/content/drive/MyDrive/Colab\ Notebooks/'

In [ ]:
!pip install --upgrade pip

# EDINET API
!pip install edinet-xbrl

# Arelle
!pip install arelle
!pip install lxml
!pip install isodate

In [3]:
# --------------------
# 各種ライブラリーのインポート
# --------------------
import os
import warnings
warnings.simplefilter('ignore')
import gc

import requests
from pprint import pprint

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100) 
pd.set_option('display.max_columns', 150)
from google.colab.data_table import DataTable
DataTable.max_columns = 300

from tqdm.notebook import tqdm
import time 
import datetime as dt

In [4]:
import datetime
import pytz

def COPY_File2Zip(Original_File_PATH, Copy_To_Directory, DELETE_ZIP=False, File_Name=None):  
  if File_Name is None:
    File_Name = Original_File_PATH.split("/")[-1] # Fold Name  
  print(f"Original_File_PATH: {Original_File_PATH}\n ==>> Copy_To_Directory: {Copy_To_Directory}")  
  print(f"*** CREATE {File_Name}.zip ", end="")  
  !zip -q -r {File_Name}.zip {Original_File_PATH}  
  print(f"*** COPY TO ", end="")  
  !cp -f /content/{File_Name}.zip {Copy_To_Directory}/{File_Name}.zip  
  if DELETE_ZIP:  
    print(f"*** DELETE Original_ZIP ", end="")  
    !rm /content/{File_Name}.zip  
  print(f"*** DONE!")

In [5]:
def reduce_mem_usage(df, verbose=True): 
    if verbose: 
        print("*** reduce_mem_usage ***") 
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64'] 
    start_mem = df.memory_usage().sum() / 1024**2 
    for col in df.columns: 
        col_type = df[col].dtypes 
        if col_type in numerics: 
            c_min = df[col].min() 
            c_max = df[col].max() 
            if str(col_type)[:3] == 'int': 
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max: 
                    df[col] = df[col].astype(np.int8) 
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max: 
                    df[col] = df[col].astype(np.int16) 
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max: 
                    df[col] = df[col].astype(np.int32) 
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max: 
                    df[col] = df[col].astype(np.int64) 
            else: 
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max: 
                    df[col] = df[col].astype(np.float16) 
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max: 
                    df[col] = df[col].astype(np.float32) 
                else: 
                    df[col] = df[col].astype(np.float64)        
#         elif col_type == object: 
#             df[col] = df[col].astype('category') 
    end_mem = df.memory_usage().sum() / 1024**2 
    if verbose: 
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem)) 
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem)) 
    gc.collect();
    return df

# EDINET API

In [6]:
# -----------------------------
# EDINET API 仕様書（Version 1）
# -----------------------------
# 3-1 参考資料: 書類種別コード
# 
docTypeCodeList = {'010':'有価証券通知書', '020':'変更通知書（有価証券通知書）', '030':'有価証券届出書',
                   '040':'訂正有価証券届出書', '050':'届出の取下げ願い', '060':'発行登録通知書', 
                   '070':'変更通知書（発行登録通知書）', '080':'発行登録書', '090':'訂正発行登録書',
                   '100':'発行登録追補書類', '110':'発行登録取下届出書', '120':'有価証券報告書',
                   '130':'訂正有価証券報告書', '135':'確認書', '136':'訂正確認書', '140':'四半期報告書',
                   '150':'訂正四半期報告書', '160':'半期報告書', '170':'訂正半期報告','180':'臨時報告書',
                   '190':'訂正臨時報告書', '200':'親会社等状況報告書', '210':'訂正親会社等状況報告書',
                   '220':'自己株券買付状況報告書', '230':'訂正自己株券買付状況報告書', '235':'内部統制報告書',
                   '236':'訂正内部統制報告書', '240':'公開買付届出書', '250':'訂正公開買付届出書',
                   '260':'公開買付撤回届出書', '270':'公開買付報告書', '280':'訂正公開買付報告書',
                   '290':'意見表明報告書', '300':'訂正意見表明報告書', '310':'対質問回答報告書',
                   '320':'訂正対質問回答報告書', '330':'別途買付け禁止の特例を受けるための申出書',
                   '340':'訂正別途買付け禁止の特例を受けるための申出書', '350':'大量保有報告書',
                   '360':'訂正大量保有報告書', '370':'基準日の届出書', '380':'変更の届出書',}

In [7]:
start_date = '2022-06-23'
end_date='2022-06-23'
docTypeCode_list=['120','140'] # 有価証券報告書 と 四半期報告書 を指定

# secCode_list=None
secCode_list=['72030'] # トヨタ

BaseDir=BaseDir
APIFileDataDir=APIDir
# DOWNLOAD_STOPPER=None
DOWNLOAD_STOPPER=5

## FUNCTION

In [8]:
def Get_EDINET_API_Data(start_date='2022-04-01', end_date='2022-04-01', docTypeCode_list=['120'], secCode_list=None,
                        BaseDir='',  APIFileDataDir='', DOWNLOAD_STOPPER=2):
  """ Get_EDINET_API_Data
  EDINET API から指定期間の提出書類一覧データ及び書類データのダウンロード
  Args:
    start_date(str):取得期間の開始日
    end_date(str):取得期間の終了日
    docTypeCode_list(list):取得する書類種別のコード
    secCode_list(list or None):取得する証券コードの指定( None ですべて取得 / 4桁株式コードの1の位に 0 の5桁?)
    BaseDir(str):ZIPをダウンロードするフォルダ
    APIFileDataDir(str):ZIPファイルの解凍先フォルダ
    DOWNLOAD_STOPPER(int or None):ダウンロード数の制限( None ですべて取得 )
  Raises:
    TypeError:取得期間の開始日より前の日付を終了日で指定した場合に発生
  Returns:
    list:提出書類一覧データ
  """

  t_start_1 = time.time()

  # 日付の処理
  start_date = pd.to_datetime(start_date, format="%Y-%m-%d")
  end_date = pd.to_datetime(end_date, format="%Y-%m-%d")
  if start_date > end_date:
    raise TypeError("'end_date' must be later than 'start_date'")
  
  # --------------------
  # 指定期間の 提出書類一覧データ を１日づつ取得
  # --------------------
  print('*'*20)
  print(f'GET API: {(end_date - start_date).days +1} DAYS({start_date.strftime("%Y-%m-%d")} to {end_date.strftime("%Y-%m-%d")}), docTypeCode: {docTypeCode_list}, secCode: {secCode_list}, DOWNLOAD_STOPPER: {DOWNLOAD_STOPPER}')
  print('*'*20)
  res=[]
  for c in tqdm(range((end_date - start_date).days +1)):
    date = start_date + dt.timedelta(days=c)
    res_ = get_date_data(date.strftime('%Y-%m-%d'), docTypeCode_list, secCode_list)
    res.extend(res_)
    time.sleep(1) # API負荷軽減 WAIT 1 Secounds

  # --------------------
  # ダウンロードする 書類データ の docID をリスト化
  # --------------------
  doc_id_list = create_doc_id_list(res)
  print(f"::: COUNT docID: {len(doc_id_list)} :::", end=''); print(f"::: ElapsedTime: {time.time() - t_start_1:.0f}s :::", end='');

  t_start_2 = time.time()
  # --------------------
  # 取得した docID の 書類データ をダウンロード
  # --------------------
  if DOWNLOAD_STOPPER == None:
    for doc_id in tqdm(set(doc_id_list)):
      requests_api(date=None, doc_id=doc_id, BaseDir=BaseDir, APIFileDataDir=APIFileDataDir)
      time.sleep(1) # API負荷軽減 WAIT 1 Secounds
  else:
    num=1
    for doc_id in tqdm(set(doc_id_list)):
      if num <= DOWNLOAD_STOPPER:
        requests_api(date=None, doc_id=doc_id, BaseDir=BaseDir, APIFileDataDir=APIFileDataDir)
        time.sleep(1) # API負荷軽減 WAIT 1 Secounds
        num+=1

  print(f"::: DOWNLOAD doc ElapsedTime: {time.time() - t_start_2:.0f}s :::", end='')
  
  print(f"::: FIN ElapsedTime: {time.time() - t_start_1:.0f}s :::", end='')
  return res

def requests_api(date=None, doc_id=None, BaseDir='', APIFileDataDir=''):
  """ requests_api
    Note:
      EDINET API 仕様書（Version 1）
      -----

      2-1-1 リクエストについて: リクエストパラメータ
      -----
      res_params:
      date: 日付（YYYY-MM-DD形式）
            出力対象とする提出書類一覧のファイル日付を指定
      type: 1 => メタデータのみを取得
            2 => 提出書類一覧及びメタデータを取得

      2-2-1 リクエストについて: リクエストパラメータ
      -----
      res_params:
      type: 1 => 提出本文書及び監査報告書を取得
    -----
    EDINET API関連資料 - EDINET API仕様書
      https://disclosure.edinet-fsa.go.jp/EKW0EZ0015.html
  """
  # --------------------
  # 書類一覧データの取得
  # --------------------
  if doc_id == None:
    end_point = 'https://disclosure.edinet-fsa.go.jp/api/v1/documents.json'
    res_params = {
        'date': date,
        'type': 2,
    }
    return requests.get(end_point, params=res_params, verify=False)

  # --------------------
  # 書類データの取得
  # --------------------
  else:
    # GET XBRL ZIP FILE
    end_point = 'https://disclosure.edinet-fsa.go.jp/api/v1/documents/' + doc_id
    res_params = {'type': 1}
    res = requests.get(end_point, params=res_params, stream=True)
    # DOWNLOAD and UNZIP
    if res.status_code == 200:
      download_and_unzip(res, doc_id, APIFileDataDir)

def download_and_unzip(requests, doc_id, APIFileDataDir):
  FileName = BaseDir + '/' + doc_id + '.zip'
  UnzipFolderName = APIFileDataDir + '/' + doc_id
  PublicDocFolder = UnzipFolderName + '/XBRL/PublicDoc'
  AuditDocFolder = UnzipFolderName + '/XBRL/AuditDoc'
  # DOWNLOAD ZIP
  with open(FileName, 'wb') as file:
    for chunk in requests.iter_content(chunk_size=1024):
      file.write(chunk)
  # UNZIP(すでに同じフォルダがあればスキップ)
  if not os.path.exists(UnzipFolderName): 
    !mkdir {UnzipFolderName}
    !unzip -q {FileName} -d {UnzipFolderName}
    # --------------------
    # 不要ファイルの削除
    # --------------------
    # 監査人フォルダは削除
    !rm -r {AuditDocFolder}
    # 画像フォルダは削除
    if os.path.exists(PublicDocFolder + '/images'):
      ImageFolder = PublicDocFolder + '/images'
      !rm -r {ImageFolder}
    # .xbr(データ本体) .xsd(タクソノミスキーマー) 以外は削除
    for _, _, files in os.walk(PublicDocFolder):
      for file in files:
        if not file.split('.')[-1] == 'xbrl' and not file.split('.')[-1] == 'xsd':
          DelFilename = PublicDocFolder + '/' + file
          !rm -r {DelFilename}
  # 元ZIP 削除
  !rm -r {FileName}

# docTypeCode_list で指定された docTypeCode の docID を取得
def get_date_data(date=None, docTypeCode_list=['120'], secCode_list=None):  
  res = requests_api(date).json()['results']
  if not res == []:
    res = selecttion_docTypeCode(res, docTypeCode_list)
    res = selecttion_secCode(res, secCode_list)
  return res
def selecttion_docTypeCode(requests=None, docTypeCode_list=['120']):
  res = []
  for request in requests:
    if request['docTypeCode'] in docTypeCode_list:
      res.append(request)
  return res
def selecttion_secCode(requests=None, secCode_list=None):
  res = []
  if secCode_list == None:
    return requests
  else:
    for request in requests:
      if request['secCode'] in secCode_list:
        res.append(request)
    return res

def create_doc_id_list(requests=[]):
  doc_id_list=[]
  for request in requests:
    doc_id_list.append(request['docID'])
  return doc_id_list

In [9]:
def create_response_api_df(response, use_col=[], APIFileDataDir=''):
  """ create_response_api_df
  Get_EDINET_API_Data で取得したXBRLファイルのパス(list)を含む response の DataFrame の作成
  Args:
    response(list):Get_EDINET_API_Data で取得した response
    use_col(list):response のうち DataFrame として返す要素
    APIFileDataDir(str):ZIPファイルの解凍先フォルダ
  Returns:
    pandas.DataFrame:
  """
  df = pd.DataFrame(response)
  df = get_xbrl_filepath(df, APIFileDataDir=APIFileDataDir)
  use_col += ['XBRL_filepaths']
  df = df[use_col]
  return df
def get_xbrl_filepath(df, APIFileDataDir=''):
  """ get_xbrl_filepath
    Note:
      Get_EDINET_API_Data でダウンロードしたXBRLフォルダを走査して .xbrl ファイルのパスを list で取得
  """
  df['XBRL_filepaths'] = APIFileDataDir + '/' + df['docID'] + '/XBRL/PublicDoc'
  for i, dir in enumerate(df['XBRL_filepaths']):
    filepaths = []
    try:
      for _, _, files in os.walk(dir):
        for file in files:
          if file.split('.')[-1] == 'xbrl':
            filepaths.append(dir+'/'+file)
      df['XBRL_filepaths'].iloc[i] = filepaths
    except:
      df['XBRL_filepaths'].iloc[i] = filepaths
  return df

## GET

In [10]:
res = Get_EDINET_API_Data(start_date=start_date, end_date=end_date,
                          docTypeCode_list=docTypeCode_list, secCode_list=secCode_list,
                          BaseDir=BaseDir,  APIFileDataDir=APIFileDataDir, DOWNLOAD_STOPPER=DOWNLOAD_STOPPER)

use_col = ['submitDateTime', 'filerName', 'secCode', 'edinetCode', 'docID', 'docTypeCode', 'docDescription', 'periodStart', 'periodEnd',]
API_data = create_response_api_df(res, use_col, APIFileDataDir=APIDir)
API_data = reduce_mem_usage(API_data)

********************
GET API: 1 DAYS(2022-06-23 to 2022-06-23), docTypeCode: ['120', '140'], secCode: ['72030'], DOWNLOAD_STOPPER: 5
********************


  0%|          | 0/1 [00:00<?, ?it/s]

::: COUNT docID: 1 :::::: ElapsedTime: 2s :::

  0%|          | 0/1 [00:00<?, ?it/s]

rm: cannot remove '/content/EDINET_API/APIdata/S100OC13/XBRL/PublicDoc/51_0529900102206.gif': No such file or directory
rm: cannot remove '/content/EDINET_API/APIdata/S100OC13/XBRL/PublicDoc/52_c_0529900102206-01.gif': No such file or directory
rm: cannot remove '/content/EDINET_API/APIdata/S100OC13/XBRL/PublicDoc/50_0529900102206.gif': No such file or directory
::: DOWNLOAD doc ElapsedTime: 13s :::::: FIN ElapsedTime: 15s :::*** reduce_mem_usage ***
Memory usage after optimization is: 0.00 MB
Decreased by 0.0%


In [11]:
API_data.head(5)

,submitDateTime,filerName,secCode,edinetCode,docID,docTypeCode,docDescription,periodStart,periodEnd,XBRL_filepaths
0,2022-06-23 15:10,トヨタ自動車株式会社,72030,E02144,S100OC13,120,有価証券報告書－第118期(令和3年4月1日－令和4年3月31日),2021-04-01,2022-03-31,[/content/EDINET_API/APIdata/S100OC13/XBRL/Pub...


# XBRL DATA LOAD

## FUNCTION

In [12]:
import re
from arelle import Cntlr

def create_XBRL_data_df(df, docID_col='docID', paths_col='XBRL_filepaths',
                        val_area=[],text_area=[],currentY_list=[],priorY_list=[]):
  """ create_XBRL_data_df
  Get_EDINET_API_Data でダウンロードした書類データとcreate_response_api_df で作成した DataFrame から
  Args:
    df(pandas.DataFrame)):create_response_api_df で作成した DataFrame
    docID_col(str):df のうち docID が入力されたカラム名
    paths_col(str):df のうち XBRLファイルのパス が入力されたカラム名
    val_area(list):XBRLから取得する数値データの日本語ラベル名
    text_area(list):XBRLから取得する文字データの日本語ラベル名
    currentY_list(list):XBRLから当年として処理する contextID
    priorY_list(list):XBRLから前年として処理する contextID
  Returns:
    pandas.DataFrame:
  Note:
    XBRLファイルの名前空間の処理には Arelle を使用
  """
  # 取得する日本語ラベル名
  label_list = val_area+text_area

  # 出力用の DataFrame の作成
  col_name = []
  label_t=['_当年','_前年',]
  for l in val_area:
    for t in label_t:
      col_name.append(l+t)
  col_name_list = col_name + text_area
  df_col = [docID_col,'file_name'] + col_name_list
  df_return = pd.DataFrame([], columns=df_col)

  # --------------------
  # XBRLファイルの読取と取得
  # --------------------
  for i in tqdm(range(len(df))):
    docID = df[docID_col].iloc[i]
    if not df[paths_col].iloc[i] == []:
      for xbrl_file in df[paths_col].iloc[i]:
        # データを追加する箱の準備(空の1行DataFrame作成)
        df_return_ = pd.DataFrame(index=[0], columns=df_col)
        # docID は df 1行あたり共通
        df_return_[docID_col] = docID
        # XBRLファイル名の取得
        file_name = xbrl_file.split('/')[-1]
        df_return_['file_name'] = file_name

        # Arelle 初期化
        ctrl = Cntlr.Cntlr(logFileName='logToPrint')
        model_xbrl = ctrl.modelManager.load(xbrl_file)

        # 名前空間を１個づつ取得
        for fact in model_xbrl.facts:
          label_ja = fact.concept.label(preferredLabel=None, lang='ja', linkroleHint=None)
          # 日本語ラベルが取得するラベルと一致したら読取処理
          if label_ja in label_list:
            # 数値データの場合に value としてデータ取得
            try:
              value = fact.vEqValue
              if type(value) == int or type(value) == float:
                contextID = fact.contextID
                if contextID in currentY_list:
                  label_ja = label_ja+'_当年'
                if contextID in priorY_list:
                  label_ja = label_ja+'_前年'
            except ValueError as e:
              pass
            
            # 文字データの場合に value としてデータ取得し、データのクリーンを実行
            try:
              value = fact.text
              # HTMLタグと改行の削除
              value = str(value).replace('\n', '')
              p = re.compile(r"<[^>]*?>")
              value = p.sub('', value)
            except ValueError as e:
              pass

            # 取得したデータの追加
            if label_ja in df_col:
              df_return_[label_ja] = value
        
        # XBRLファイルを1つ処理したら返値用の DataFrame に行を追加
        df_return = pd.concat([df_return, df_return_], axis=0)
  
  df_return.reset_index(inplace=True, drop=True)
  return df_return

## SETUP

In [13]:
val_area_BS = ['流動資産','固定資産','資産',
               '流動負債','固定負債','負債',
               '資本金','株主資本','利益剰余金','純資産','負債純資産',]

In [14]:
Earnings = ['売上高','売上収益','売上原価',]
Earnings += ['営業収益','営業収入','営業総収入','営業費用','営業原価',]
Earnings += ['金融収益','金融費用',]
Earnings += ['鉄軌道事業営業収益','鉄軌道事業営業費',]
Earnings += ['海運業収益及びその他の営業収益','海運業費用及びその他の営業費用',]
Earnings += ['事業収益','事業費用',]
Earnings += ['教育研究・附属事業収益','教育研究・附属事業費用','収益事業収益','収益事業費用',]
Gross_profit = ['売上総利益又は売上総損失（△）',]
Gross_profit += ['純営業収益',]
Gross_profit += ['営業総利益又は営業総損失（△）',]
Gross_profit += ['事業利益又は事業損失（△）',]

Expenses = ['販売費及び一般管理費',]
Expenses += ['一般管理費',]
Expenses += ['供給販売費及び一般管理費',]
Operating_profit = ['営業利益又は営業損失（△）',]
Operating_profit += ['全事業営業利益又は全事業営業損失（△）',]

Non_operating = ['営業外収益','営業外費用',]
Non_operating += ['経常収益','経常費用',]
Non_operating += ['事業外収益','事業外費用',]
Ordinary_profit = ['経常利益又は経常損失（△）',]

Income_before_tax = ['特別利益','特別損失','契約者配当準備金繰入額','税引前当期純利益又は税引前当期純損失（△）',]

Net_income = ['法人税等','当期純利益又は当期純損失（△）',]

val_area_PL = Earnings+Gross_profit+Expenses+Operating_profit+Non_operating+Ordinary_profit+Income_before_tax+Net_income

In [15]:
text_area = ['連結経営指標等','提出会社の経営指標等',
             '沿革','事業の内容','関係会社の状況','従業員の状況',
             '経営方針、経営環境及び対処すべき課題等','事業等のリスク',
             '経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析',
             '経営上の重要な契約等','研究開発活動','設備投資等の概要',
             '大株主の状況','コーポレート・ガバナンスの概要','追加情報',
             'セグメント情報等','報告セグメントの概要','重要な後発事象',]

In [16]:
# 未実装の注意 各年の時点(Instant/Duration, Year/Interim/YTD/Quarter )が複数ある場合には対応していない(単一の報告書にはどれか１つだけが存在している想定)
currentY_list = ['CurrentYear','Interim','CurrentYTD','CurrentQuarter',]
priorY_list = ['Prior1Year','Prior1Interim','Prior1YTD','Prior1Quarter',]

l=[]
for c in currentY_list:
  for t in ['Instant','Duration',]:
    for m in ['','_NonConsolidatedMember']:
      l.append(c+t+m)
currentY_list = l

l=[]
for c in priorY_list:
  for t in ['Instant','Duration',]:
    for m in ['','_NonConsolidatedMember']:
      l.append(c+t+m)
priorY_list = l

#### タクソノミ調査_作業用

**Arelleを用いない場合**
> 解読に使用するソース  
> URL: [EDINETタクソノミ](https://www.fsa.go.jp/search/index.html) - YYYY年版EDINETタクソノミ  

３．公表資料  
* (1). EDINETタクソノミ関連  
名前空間プレフィックス 及び 要素名 **(qnameに該当)**
 * (e) タクソノミ要素リスト
 * (f) 勘定科目リスト
* (2). XBRL作成ガイド
5-4 コンテキストの定義 **(contextID に該当)**
 * (b)	報告書インスタンス作成ガイドライン  

In [17]:
# import re
# from arelle import Cntlr

# xbrl_file = '/content/EDINET_API/APIdata/S100OC13/XBRL/PublicDoc/jpcrp030000-asr-001_E02144-000_2022-03-31_01_2022-06-23.xbrl'

# ctrl = Cntlr.Cntlr(logFileName='logToPrint')
# model_xbrl = ctrl.modelManager.load(xbrl_file)

# for fact in model_xbrl.facts:
#   text_area=False
#   # --------------------
#   # 情報の取得
#   # --------------------
#   # 日本語ラベル
#   label_ja = fact.concept.label(preferredLabel=None, lang='ja', linkroleHint=None)
#   # 英語ラベル
#   label_en = fact.concept.label(preferredLabel=None, lang='en', linkroleHint=None)
#   # qname
#   qname = fact.qname
#   try:
#     namespace_prefix = str(qname).split(':')[0]
#     element_name = str(qname).split(':')[-1]
#   except ValueError as e:
#     namespace_prefix = qname
#     element_name = qname
#   # contextID
#   contextID = fact.contextID
#   # 単位
#   if fact.unit is None:
#     unit = None
#   else:
#     unit = fact.unit.value
#   # Value
#   try:
#     # 数値
#     value = fact.xValue
#   except ValueError as e:
#     pass
#   try:
#     # 文字列
#     value = fact.text
#     # HTMLタグと改行の削除
#     value = str(value).replace('\n', '')
#     p = re.compile(r"<[^>]*?>")
#     value = p.sub('', value)
#     text_area = True
#   except ValueError as e:
#     pass
  
#   # print('日本語ラベル:', label_ja)
#   # print('英語ラベル:', label_en)
#   # print('qname:', qname)
#   # print('名前空間プレフィックス:', namespace_prefix)
#   # print('要素名:', element_name)
#   # print('contextID:', contextID)
#   # print('単位:', unit)
#   # print('Value:', value)
#   # print('*'*20)
#   if text_area:
#     print('日本語ラベル:', label_ja)
#     print('Value:', value)
#     print('*'*20)
#   # if label_ja == '大株主の状況':
#   #   print('日本語ラベル:', label_ja)
#   #   print('英語ラベル:', label_en)
#   #   print('qname:', qname)
#   #   print('名前空間プレフィックス:', namespace_prefix)
#   #   print('要素名:', element_name)
#   #   print('contextID:', contextID)
#   #   print('単位:', unit)
#   #   print('Value:', value)
#   #   print('*'*20)

## GET

In [18]:
paths_col='XBRL_filepaths'
docID_col='docID'

val_area=val_area_BS
text_area_=[]
XBRL_BS_data =  create_XBRL_data_df(API_data, docID_col=docID_col, paths_col=paths_col,
                                    val_area=val_area, text_area=text_area_,
                                    currentY_list=currentY_list, priorY_list=priorY_list,)
XBRL_BS_data = reduce_mem_usage(XBRL_BS_data)

val_area=val_area_PL
text_area_=text_area
XBRL_PL_data =  create_XBRL_data_df(API_data, docID_col=docID_col, paths_col=paths_col,
                                    val_area=val_area, text_area=text_area_,
                                    currentY_list=currentY_list, priorY_list=priorY_list,)
XBRL_PL_data = reduce_mem_usage(XBRL_PL_data)

  0%|          | 0/1 [00:00<?, ?it/s]

*** reduce_mem_usage ***
Memory usage after optimization is: 0.00 MB
Decreased by 0.0%


  0%|          | 0/1 [00:00<?, ?it/s]

*** reduce_mem_usage ***
Memory usage after optimization is: 0.00 MB
Decreased by 0.0%


In [19]:
API_BS_data = pd.merge(API_data, XBRL_BS_data, on='docID', how='right')
API_BS_data.drop('XBRL_filepaths',axis=1).to_csv(f'{OutputDir}/EDINET_API_BS_DATA.csv', encoding='cp932',errors="ignore")

In [20]:
API_PL_data = pd.merge(API_data, XBRL_PL_data, on='docID', how='right')
API_PL_data.drop('XBRL_filepaths',axis=1).to_csv(f'{OutputDir}/EDINET_API_PL_DATA.csv', encoding='cp932',errors="ignore")

In [21]:
# XBRL_BS_data

In [22]:
# XBRL_PL_data

In [23]:
API_BS_data

,submitDateTime,filerName,secCode,edinetCode,docID,docTypeCode,docDescription,periodStart,periodEnd,XBRL_filepaths,file_name,流動資産_当年,流動資産_前年,固定資産_当年,固定資産_前年,資産_当年,資産_前年,流動負債_当年,流動負債_前年,固定負債_当年,固定負債_前年,負債_当年,負債_前年,資本金_当年,資本金_前年,株主資本_当年,株主資本_前年,利益剰余金_当年,利益剰余金_前年,純資産_当年,純資産_前年,負債純資産_当年,負債純資産_前年
0,2022-06-23 15:10,トヨタ自動車株式会社,72030,E02144,S100OC13,120,有価証券報告書－第118期(令和3年4月1日－令和4年3月31日),2021-04-01,2022-03-31,[/content/EDINET_API/APIdata/S100OC13/XBRL/Pub...,jpcrp030000-asr-001_E02144-000_2022-03-31_01_2...,8340434000000,8819286000000,12650607000000,12378995000000,20991040000000,21198281000000,4798256000000,5702195000000,1585512000000,1602065000000,6383768000000,7304260000000,635402000000,635402000000,13046366000000,12606029000000,15066843000000,14462819000000,14607272000000,13894021000000,20991040000000,21198281000000


In [24]:
API_PL_data

,submitDateTime,filerName,secCode,edinetCode,docID,docTypeCode,docDescription,periodStart,periodEnd,XBRL_filepaths,file_name,売上高_当年,売上高_前年,売上収益_当年,売上収益_前年,売上原価_当年,売上原価_前年,営業収益_当年,営業収益_前年,営業収入_当年,営業収入_前年,営業総収入_当年,営業総収入_前年,営業費用_当年,営業費用_前年,営業原価_当年,営業原価_前年,金融収益_当年,金融収益_前年,金融費用_当年,金融費用_前年,鉄軌道事業営業収益_当年,鉄軌道事業営業収益_前年,鉄軌道事業営業費_当年,鉄軌道事業営業費_前年,海運業収益及びその他の営業収益_当年,海運業収益及びその他の営業収益_前年,海運業費用及びその他の営業費用_当年,海運業費用及びその他の営業費用_前年,事業収益_当年,事業収益_前年,事業費用_当年,事業費用_前年,教育研究・附属事業収益_当年,教育研究・附属事業収益_前年,教育研究・附属事業費用_当年,教育研究・附属事業費用_前年,収益事業収益_当年,収益事業収益_前年,収益事業費用_当年,収益事業費用_前年,売上総利益又は売上総損失（△）_当年,売上総利益又は売上総損失（△）_前年,純営業収益_当年,純営業収益_前年,営業総利益又は営業総損失（△）_当年,営業総利益又は営業総損失（△）_前年,事業利益又は事業損失（△）_当年,事業利益又は事業損失（△）_前年,販売費及び一般管理費_当年,販売費及び一般管理費_前年,一般管理費_当年,一般管理費_前年,供給販売費及び一般管理費_当年,供給販売費及び一般管理費_前年,営業利益又は営業損失（△）_当年,営業利益又は営業損失（△）_前年,全事業営業利益又は全事業営業損失（△）_当年,全事業営業利益又は全事業営業損失（△）_前年,営業外収益_当年,営業外収益_前年,営業外費用_当年,営業外費用_前年,経常収益_当年,経常収益_前年,経常費用_当年,経常費用_前年,事業外収益_当年,事業外収益_前年,事業外費用_当年,事業外費用_前年,経常利益又は経常損失（△）_当年,経常利益又は経常損失（△）_前年,特別利益_当年,特別利益_前年,特別損失_当年,特別損失_前年,契約者配当準備金繰入額_当年,契約者配当準備金繰入額_前年,税引前当期純利益又は税引前当期純損失（△）_当年,税引前当期純利益又は税引前当期純損失（△）_前年,法人税等_当年,法人税等_前年,当期純利益又は当期純損失（△）_当年,当期純利益又は当期純損失（△）_前年,連結経営指標等,提出会社の経営指標等,沿革,事業の内容,関係会社の状況,従業員の状況,経営方針、経営環境及び対処すべき課題等,事業等のリスク,経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析,経営上の重要な契約等,研究開発活動,設備投資等の概要,大株主の状況,コーポレート・ガバナンスの概要,追加情報,セグメント情報等,報告セグメントの概要,重要な後発事象
0,2022-06-23 15:10,トヨタ自動車株式会社,72030,E02144,S100OC13,120,有価証券報告書－第118期(令和3年4月1日－令和4年3月31日),2021-04-01,2022-03-31,[/content/EDINET_API/APIdata/S100OC13/XBRL/Pub...,jpcrp030000-asr-001_E02144-000_2022-03-31_01_2...,12607858000000,11761405000000,NaN,NaN,10295206000000,9939958000000,NaN,NaN,NaN,NaN,NaN,NaN,28383811000000,25016845000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2312652000000,1821447000000,NaN,NaN,NaN,NaN,NaN,NaN,1182963000000,1122074000000,NaN,NaN,NaN,NaN,1129689000000,699373000000,NaN,NaN,1151431000000,1276645000000,109591000000,89327000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2171530000000,1886691000000,NaN,NaN,NaN,NaN,NaN,NaN,2171530000000,1886691000000,477583000000,248634000000,1693947000000,1638057000000,(１) 連結経営指標等 回 次国際財務報告基準移行日第116期第117期第118期決算期20...,(２) 提出会社の経営指標等 回次第114期第115期第116期第117期第118期決算期...,２ 【沿革】 年月概要1933年９月㈱豊田自動織機製作所（現在の㈱豊田自動織機）内で自動車...,３ 【事業の内容】連結財務諸表提出会社（以下、当社という。）は、IFRSに準拠して連結財務...,４ 【関係会社の状況】 名称住所資本金又は出資金主要な事業の内容議決権の所有割合(％...,５ 【従業員の状況】（１）連結会社の状況2022年３月31日現在事業別セグメントの名称従業...,１ 【経営方針、経営環境及び対処すべき課題等】本項においては、将来に関する事項が含まれていま...,２ 【事業等のリスク】以下において、トヨタの事業その他のリスクについて、投資家の判断に重要...,３ 【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】（１）経営成績等の状...,４ 【経営上の重要な契約等】2002年１月チェコ共和国において小型乗用車を共同生産するため、...,５ 【研究開発活動】当社は、「クリーンで安全な商品の提供を使命とし、あらゆる企業活動を通じ...,１ 【設備投資等の概要】トヨタでは、投資効率の向上をはかりつつ、環境問題などの社会的要請に対...,(６) 【大株主の状況】2022年３月31日現在氏名又は名称住所所有株式数(千株)発行済株...,(１) 【コーポレート・ガバナンスの概要】①コーポレート・ガバナンスに関する基本的な考え方当...,NaN,NaN,NaN,（重要な後発事象） １ 自己株式取得 当社は、2022年５月11日開催の取締役会において、...


## DATA UPLOAD GOOGLE DRIVE

In [25]:
if GoogleDriveMount:

  dt_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  dttime = str(dt_now.year)+str(dt_now.month)+str(dt_now.day)+"_"+str(dt_now.hour)+"_"+str(dt_now.minute)+"_"+str(dt_now.second) 
  File_Name=f"{dttime}"

  Original_File_PATH = OutputDir
  Copy_To_Directory = CopyGoogleDriveDir

  COPY_File2Zip(Original_File_PATH, Copy_To_Directory, DELETE_ZIP=False, File_Name=File_Name)

# 参考

*   ドキュメント
 *   [EDINET API仕様書](https://disclosure.edinet-fsa.go.jp/EKW0EZ0015.html)
 *   [公表済のEDINETタクソノミ](https://www.fsa.go.jp/search/index.html)

*   実装関連  
 *   [Pythonを使ってEdinetから四半期報告書をダウンロードする方法](https://qlitre-weblog.com/how-to-get-financial-infomation-using-python/)
 *   [EDINET XBRLを読み込む方法【Python】](https://srbrnote.work/archives/1236)
 *   [【Python】XBRL から『勘定科目』と『リンクベース』の内容を取得するコード例【Arelle】](https://srbrnote.work/archives/5611)
 *   [XBRLタクソノミの表示リンクをOSSのArrele APIを使ってたどってみよう](https://qiita.com/XBRLJapan/items/856cd4504b316f18b8b5)



